## DL Text Classification

Пайплайн взят из примеров торча для текстовой классификации.

In [ ]:
# !pip install torchmetrics

In [86]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torchtext import data
from torch.utils.data import DataLoader

import torch.nn as nn
from torchtext.vocab import build_vocab_from_iterator
import time

from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchmetrics.classification import Precision

In [2]:
df = pd.read_csv('preprocessed_texts.csv')
df.head()

,Unnamed: 0,text,label,text_lemmatized
0,0,why don’t trade your btc to earn weekly rathe...,1,"['dont', 'trade', 'btc', 'earn', 'weekly', 'ra..."
1,1,"i’ll be honest, i didn’t see it sinking this ...",1,"['ill', 'honest', 'didnt', 'see', 'sinking', '..."
2,2,-x needs to break 38 to hit 4 no shit lol,1,"['x', 'need', 'break', 'hit', 'shit', 'lol']"
3,3,it's gonna be an all nighter ha?,1,"['gon', 'na', 'nighter', 'ha']"
4,4,y’all should just come to cro.x,2,"['yall', 'come', 'crox']"


In [3]:
df['text_lemmatized'].values

array(["['dont', 'trade', 'btc', 'earn', 'weekly', 'rather', 'hodl']",
       "['ill', 'honest', 'didnt', 'see', 'sinking', 'low', 'fast', 'usually', 'dump', 'got', 'least', 'one', 'day', 'stroke', 'bounce', 'back', 'ouch']",
       "['x', 'need', 'break', 'hit', 'shit', 'lol']", ...,
       "['im', 'happy', 'af', 'pre', 'black', 'friday', 'event']",
       "['new', 'article', 'exactly', 'cardano', 'ada', 'get', 'fud']",
       "['bollinger', 'band', 'rsi', 'hr', 'big', 'pump', 'coming', 'manipulate', 'u', 'sooner', 'think']"],
      dtype=object)

In [4]:
# def make_lst(strng):
#     s = []
#     strng = strng[1:-1].split(', ')
#     for el in strng:
#         s.append(el[1:-1])
#     return s

In [5]:
# df.text_lemmatized = df.text_lemmatized.apply(make_lst)

In [31]:
X, y = df['text_lemmatized'].values, df['label'].values

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=228)

In [33]:
np_train = np.dstack((y_train, X_train))[0]
np_train[0]

array([2,
       list(['fjb', 'take', 'back', 'america', 'vote', 'republican', 'fight', 'back', 'crime'])],
      dtype=object)

In [34]:
np_test = np.dstack((y_test, X_test))[0]
np_test[0]

array([1, list(['fu', 'anyone', 'happy', 'made', 'profitseriously'])],
      dtype=object)

Создаем итераторы данных для DataLoader

In [81]:
train_iter = iter((tuple(el) for el in np_train))

In [83]:
test_iter = iter((tuple(el) for el in np_test))

In [12]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield text

In [27]:
# Создаем словарь слов из токенов - кодируются числами
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [28]:
vocab(['ok', 'made', 'dough', 'goin', 'buy', 'sovereign', 'debt', 'really', 'smh'])

[357, 189, 6039, 1745, 1, 4899, 1100, 96, 1107]

In [15]:
# Пайплайны для предобрабработки данных (в будущем можно дополнить токенизатором из torch в случае сырых данных)
text_pipeline = lambda x: vocab(x)
label_pipeline = lambda x: int(x)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    '''
    Функция для обработки батчей данных
    '''
    label_list, text_list, offsets = [], [], [0] # offsets - начальные индексы отдельных элементов в тексте 
    for (label, text) in batch:
         label_list.append(label_pipeline(label))
         processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

Будем использовать модель с EmbeddingBag слоем - он добавляет ключевое слово к тексту и не преобразует текст каким-либо другим способом, достаточно эффективен на коротких предложениях.

In [17]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [18]:
num_class = 3
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [80]:
def train(dataloader):
    model.train()
    total_pr, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        precision = Precision(task="multiclass", average='micro', num_classes=3)
        total_pr += precision(predicted_label.argmax(1), label)
        total_count += 1
      
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| precision {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_pr/total_count))
            total_pr, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_pr, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            precision = Precision(task="multiclass", average='micro', num_classes=3)
            total_pr += precision(predicted_label.argmax(1), label)
            total_count += 1
    return total_pr/total_count

In [84]:
EPOCHS = 10 
LR = 5 
BATCH_SIZE = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# создадим валидационную выборку
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid pr {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/13787 batches | precision    0.574
| epoch   1 |  1000/13787 batches | precision    0.572
| epoch   1 |  1500/13787 batches | precision    0.571
| epoch   1 |  2000/13787 batches | precision    0.578
| epoch   1 |  2500/13787 batches | precision    0.579
| epoch   1 |  3000/13787 batches | precision    0.573
| epoch   1 |  3500/13787 batches | precision    0.570
| epoch   1 |  4000/13787 batches | precision    0.572
| epoch   1 |  4500/13787 batches | precision    0.575
| epoch   1 |  5000/13787 batches | precision    0.572
| epoch   1 |  5500/13787 batches | precision    0.573
| epoch   1 |  6000/13787 batches | precision    0.573
| epoch   1 |  6500/13787 batches | precision    0.570
| epoch   1 |  7000/13787 batches | precision    0.573
| epoch   1 |  7500/13787 batches | precision    0.577
| epoch   1 |  8000/13787 batches | precision    0.572
| epoch   1 |  8500/13787 batches | precision    0.576
| epoch   1 |  9000/13787 batches | precision    0.569
| epoch   

In [85]:
test_precision = evaluate(test_dataloader)
print('test precision {:8.3f}'.format(test_precision))

test precision    0.582


Метрика precision получилась сравнима с оптимизацией CatBoost из прошлого чекпоинта. Напомним, что предположительно качество остается низким из-за некачественной разметки датасета.